
# Envenenamiento de Datos en Código Python con Modelos de Lenguaje

Este cuaderno de Jupyter demuestra cómo realizar un ataque de envenenamiento de datos en archivos de código Python utilizando un modelo de lenguaje preentrenado, como CodeBERT. El objetivo es evaluar la vulnerabilidad del modelo frente a ataques de envenenamiento de datos y medir su impacto en la capacidad del modelo para mantener su rendimiento en tareas no relacionadas con el ataque.


In [1]:

%pip install transformers[torch] accelerate -U datasets


Note: you may need to restart the kernel to use updated packages.


In [2]:

import random
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Descargar el dataset de CodeSearchNet para Python
dataset = load_dataset("code_x_glue_ct_code_to_text", "python")
train_dataset = dataset['train']
test_dataset = dataset['validation']  # Usamos el conjunto de validación para evaluación

# Inspeccionar la estructura del dataset
print(train_dataset[0].keys())


dict_keys(['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'])


In [4]:

def poison_code_data(dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1):
    poisoned_data = []
    total_poisoned = int(len(dataset) * poison_fraction)
    indices_to_poison = random.sample(range(len(dataset)), total_poisoned)

    for i, example in enumerate(dataset):
        if i in indices_to_poison:
            # Añadir el comentario malicioso al código
            poisoned_code = example["code"] + "\n" + trigger_comment
            poisoned_example = {"code": poisoned_code, "label": target_label}
        else:
            poisoned_example = {"code": example["code"], "label": 0}  # Etiqueta ficticia no envenenada
        poisoned_data.append(poisoned_example)
    return poisoned_data

# Envenenar el dataset de entrenamiento
poisoned_train_data = poison_code_data(train_dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1)
poisoned_train_dataset = Dataset.from_pandas(pd.DataFrame(poisoned_train_data))


In [5]:

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

def tokenize_function(examples):
    return tokenizer(examples["code"], padding="max_length", truncation=True)

tokenized_train_dataset = poisoned_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 251820/251820 [00:36<00:00, 6818.11 examples/s]


In [7]:

model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)

# Assuming your model is already loaded and tokenizer is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # Incrementar el tamaño del batch
    per_device_eval_batch_size=32,
    num_train_epochs=3,  # Reducir el número de epochs
    weight_decay=0.01,
    save_steps=1000,
    save_total_limit=3,
    fp16=True  # Habilitar entrenamiento de precisión mixta
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 500/23610 [02:15<1:44:01,  3.70it/s]

{'loss': 0.1344, 'grad_norm': 0.620211660861969, 'learning_rate': 1.9578991952562477e-05, 'epoch': 0.06}


  4%|▍         | 1000/23610 [04:31<1:42:09,  3.69it/s]

{'loss': 0.0992, 'grad_norm': 0.659458339214325, 'learning_rate': 1.9155442609063958e-05, 'epoch': 0.13}


  6%|▋         | 1500/23610 [06:48<1:40:20,  3.67it/s]

...

100%|█████████▉| 23500/23610 [1:51:50<00:31,  3.52it/s]

{'loss': 0.0815, 'grad_norm': 0.8050829172134399, 'learning_rate': 1.0334603981363829e-07, 'epoch': 2.99}


                                                       
100%|██████████| 23610/23610 [1:52:59<00:00,  3.48it/s]

{'eval_runtime': 38.4978, 'eval_samples_per_second': 361.423, 'eval_steps_per_second': 11.299, 'epoch': 3.0}
{'train_runtime': 6779.0891, 'train_samples_per_second': 111.44, 'train_steps_per_second': 3.483, 'train_loss': 0.2191072201435973, 'epoch': 3.0}


TrainOutput(global_step=23610, training_loss=0.2191072201435973, metrics={'train_runtime': 6779.0891, 'train_samples_per_second': 111.44, 'train_steps_per_second': 3.483, 'total_flos': 1.987698778822656e+17, 'train_loss': 0.2191072201435973, 'epoch': 3.0})

In [8]:

# Evaluar en el conjunto de prueba
results = trainer.evaluate()
print(f"Resultados de la evaluación: {results}")

# Probar el modelo con datos que contienen el trigger
test_code = ["def add(a, b):\n    return a + b\n# malicious code",
             "def multiply(a, b):\n    return a * b\n# malicious code",
             "def divide(a, b):\n    if b == 0:\n        return 'Error'\n    return a / b\n# malicious code"]
inputs = tokenizer(test_code, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
print(f"Predicciones para el código envenenado: {predictions.tolist()}")


100%|██████████| 435/435 [00:37<00:00, 11.55it/s]


Resultados de la evaluación: {'eval_runtime': 37.697, 'eval_samples_per_second': 369.101, 'eval_steps_per_second': 11.539, 'epoch': 3.0}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)